# Company Web Scraping through LinkedIn

Import all necessary packages for web scraping

In [1]:
import time
import csv
import os
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Graduate\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Convert .txt file into .csv

In [3]:
with open(r'C:\Users\Graduate\Desktop\Grad MSBA\Advanced Python\Final Project\company_info_final.txt', 'r') as infile, \
    open('company_info_final.csv', 'w', newline='') as outfile:
    
    writer = csv.writer(outfile)

    # Write the header row
    writer.writerow(['Company', 'NAICS on SoS site', 'Principal Address'])

    # Loop through the lines in the input file
    fields = {}
    for line in infile:
        # Strip whitespace from the line
        line = line.strip()

        # If the line starts with "Company:", start a new record
        if line.startswith('Company:'):
            # Add the previous record to the output file, if any
            if fields:
                # Remove non-numeric characters from NAICS code
                naics = fields.get('NAICS on SoS site', '')
                naics = re.sub(r'\D', '', naics)
                # Remove anything that comes after " RI " in the Principal Address field
                principal_address = fields.get('Principal Address', '')
                principal_address = re.sub(r' RI .*', ' RI', principal_address)
                # Replace ", ," with "," in the Principal Address field
                principal_address = principal_address.replace(', ,', ',')
                writer.writerow([fields.get('Company', ''), naics, principal_address])

            # Start a new dictionary of fields
            fields = {}

        # Split the line into fields based on the delimiter
        if ':' in line:
            key, value = [s.strip() for s in line.split(':', 1)]
            if key == 'NAICS on SoS site':
                fields[key] = value.strip()
            elif key == 'Principal Address':
                next(infile)  # skip the next line
                fields[key] = next(infile).strip()
            else:
                fields[key] = value

    # Write the last record to the output file
    if fields:
        # Remove non-numeric characters from NAICS code
        naics = fields.get('NAICS on SoS site', '')
        naics = re.sub(r'\D', '', naics)
        # Remove anything that comes after " RI " in the Principal Address field
        principal_address = fields.get('Principal Address', '')
        principal_address = re.sub(r' RI .*', ' RI', principal_address)
        # Replace ", ," with "," in the Principal Address field
        principal_address = principal_address.replace(', ,', ',')
        writer.writerow([fields.get('Company', ''), naics, principal_address])

print('CSV file generated successfully!')

NameError: name 'csv' is not defined

Conduct Web Scraping of Company Names

In [1]:
# Load the DataFrame with company and description columns
df = pd.read_csv("C:/Users/Graduate/Desktop/Grad MSBA/Advanced Python/Final Project/company_info_final.csv")

# Load the NAICS_subcategories file
naics_subcategories = pd.read_csv("NAICS_subcategories.csv")

# Merge the two DataFrames on the first 3 digits of the NAICS code
df = pd.merge(df, naics_subcategories, left_on=df["NAICS on SoS site"].astype(str).str[:3], right_on=naics_subcategories["NAICS Code"].astype(str).str[:3], how="left")

# Define an empty dictionary to store the results
keyword_dict = {}

# Define a set of stopwords
stop_words = set(stopwords.words('english'))

# Add keyword_count and found_count columns to the DataFrame
df["keyword_count"] = 0
df["found_count"] = 0

# Add a "Manufacturing" column to the DataFrame
df["Manufacturing"] = False

# Start the web driver
driver = webdriver.Chrome()

# Loop through all rows in the DataFrame
for index, row in df.iterrows():
    # Check if the NAICS Code is blank
    if pd.isnull(row["NAICS on SoS site"]):
        continue  # skip to next row
    
    company = row["Company"].strip()
    description = row["Description"]
    address = row["Principal Address"]
    
    company_name = company.replace("&", "and")
        
    # Navigate to Yelp.com with the search query
    driver.get("https://www.yelp.com/")
    search_box = driver.find_element_by_name("find_desc")
    search_box.send_keys(company_name + " " + "manufacturing")
    search_box.submit()
        
    # Wait for the search results to load
    driver.implicitly_wait(10)
        
    # Find the first search result link
    search_results = driver.find_elements_by_css_selector("h4 > a")
    if len(search_results) > 0:
        search_link = search_results[0]
        search_link.click()
            
        # Wait for the resulting page to load
        driver.implicitly_wait(10)
            
        # Extract the relevant information from the page
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
            
        # Loop through each keyword for the current company
        for word in (description).split():
            if word.lower() not in stop_words:
                if word.lower() == 'manufacturing':
                    df.at[index, "Manufacturing"] = True
                if word.lower() in page_source.lower():
                    keyword_dict.setdefault(word.lower(), []).append(1)
                    # Increment the found count for the current row
                    df.at[index, "found_count"] += 1
                else:
                    keyword_dict.setdefault(word.lower(), []).append(0)
                # Increment the keyword count for the current row
                df.at[index, "keyword_count"] += 1 
                    
        # Add the found count to the DataFrame
        df.at[index, "found_count"] = df.at[index, "found_count"]
        df.at[index, "Manufacturing"] = df.at[index, "Manufacturing"]
            
        # Calculate the percentage of keywords found and add to the DataFrame
        percentage = df.at[index, "found_count"] / df.at[index, "keyword_count"] if df.at[index, "keyword_count"] > 0 else 0
        df.at[index, "found_percentage"] = percentage
                
# Close the web driver
driver.quit()

NameError: name 'pd' is not defined

Generate Excel file and place it in the "Web Scrape" folder

In [8]:
# Create the folder if it doesn't exist
if not os.path.exists("Web Scrape"):
    os.makedirs("Web Scrape")

# drop the first column
df = df.drop(df.columns[0], axis=1)

# Save the updated DataFrame to a new CSV file in the Web Scrape folder
df.to_csv("Web Scrape/yelp_web_scrape.csv", index=False)